In [2]:
debug = True

In [17]:
import termextract.janome
import termextract.core
import collections
from janome.tokenizer import Tokenizer
from pprint import pprint # このサンプルでの処理結果の整形表示のため

#上位c個の重要複合語のリストを返します.c=-1(デフォルト)ですべての複合語を返します.
def Extract_ImportantWords(text,c=-1):
    #ファイル読み込みから、テキストファイル（元の文章から抽出に変更）
    """
    f = open(filename, "r")
    text = f.read()
    f.close
    if(debug):print(text)
    """
    
    t = Tokenizer()
    tokenize_text = t.tokenize(text)
    
    #頻度ベクトル生成
    frequency = termextract.janome.cmp_noun_dict(tokenize_text) #複合語抽出
    
    #LR(ボトムアップ構文解析)のスコアの生成
    lr = termextract.core.score_lr(
        frequency,
        ignore_words=termextract.mecab.IGNORE_WORDS,
        lr_mode=1, average_rate=1)
    #頻度lRスコアの生成(すなわち重要度)
    term_imp = termextract.core.term_importance(frequency, lr)

    #降順に表示
    imp_words = []
    data_collection = collections.Counter(term_imp)
    if(c == -1):
        for cmp_noun, value in data_collection.most_common() :
            imp_words.append(termextract.core.modify_agglutinative_lang(cmp_noun))
    else:
        tmp = 1
        for cmp_noun, value in data_collection.most_common():
            if(tmp > c): break
            imp_words.append(termextract.core.modify_agglutinative_lang(cmp_noun))
            tmp += 1
    if(debug):print(imp_words)
    return imp_words

In [4]:
import requests
from bs4 import BeautifulSoup

# User-Agent

user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/83.0.4103.116 Safari/537.36 Edg/83.0.478.64"

def get_html(url, params=None, headers=None):
    """ get_html
    url: データを取得するサイトのURL
    [params]: 検索サイトのパラメーター {x: param}
    [headers]: カスタムヘッダー情報
    """
    try:
        # データ取得
        resp = requests.get(url, params=params, headers=headers)
        resp.encoding = 'utf-8'
        # 要素の抽出
        soup = BeautifulSoup(resp.text, "html.parser")
        return soup
    except Exception as e:
        return None

def get_search_url(word):
    """ get_search_url
    word: 検索するワード
    """
    try:
        # google 検索
        search_url = "https://www.google.co.jp/search"
        search_params = {"q": word}
        search_headers = {"User-Agent": user_agent}
        # データ取得
        soup = get_html(search_url, search_params, search_headers)
        if soup != None:
            #class = r <a href>
            tags = soup.select(".r > a")
            urls = [tag.get("href") for tag in tags]
            return urls
        else:
                return None
    except Exception as e:
        return None

In [5]:
def extract_textFhtml(soup):
    for script in soup(["script", "style"]):
        script.decompose()
    text=soup.get_text()
    lines=[]
    for line in text.splitlines():
        lines.append(line.strip())
    text="".join(line for line in lines if line)
    return text

In [12]:
def CreateAdditionalExplanation(filename):
    additionals = []
    importances = Extract_ImportantWords(filename) #重要言葉の抽出
    for word in importances:
        try:
            urls = get_search_url(word)
            if(urls[0] != None):
                soup = get_html(urls[0])
                text = extract_textFhtml(soup)
                additionals.append([word,text])
        except Exception as e:
            if(debug):print("エラー")
    return additionals

In [13]:
print(CreateAdditionalExplanation("out.txt"))

作ることがいかに大事かということがまああのわかるかと思います。あとは、これは中国語さっきまでできたのは英語の文法誤り訂正、何とか中国語とかロシア語だったんですが、これは中国語の誤り訂正で、でえーとまあ、単語ベクトルと言うものを三回位前やりましたよねと二勝で作ったと思うんですけど、最近、自然言語するで使われている単語ベクトルを作る手法は、文脈を考慮した単語ベクトルを作る手法が研究されていますと。
出ただ、これの入力の単語ベクトルをどういうふうに誤り訂正のシステムに入れるといいのかということが、最近までに解決の話題になっていて、ちょうど今年の今日か明日からうちの研究室のじいさんの金子くんという人がやった研究が投稿した当時、やばえっと世界最高精度だったんですけど。

['単語ベクトル', '誤り訂正', '中国語', '文法誤り訂正', '手法', '世界最高精度', '中国語', 'ロシア語', '研究室', '研究', 'あと', '英語', '位', '文脈', '入力', 'システム', '解決', '話題', 'じいさん', '金子', '人']
['単語ベクトル', '誤り訂正', '中国語', '文法誤り訂正', '手法', '世界最高精度', '中国語', 'ロシア語', '研究室', '研究', 'あと', '英語', '位', '文脈', '入力', 'システム', '解決', '話題', 'じいさん', '金子', '人']
[['単語ベクトル', 'Word2Vec：発明した本人も驚く単語ベクトルの驚異的な力 - DeepAge人工知能の今と一歩先を発信するメディアMENUすべてNumpyビッグデータ機械学習人工知能ディープラーニングTensorFlowホーム >ビッグデータ > Word2Vec：発明した本人も驚く単語ベクトルの驚異的な力採用情報アルバイト募集2016-09-2Word2Vec：発明した本人も驚く単語ベクトルの驚異的な力TweetWord2VecとはWord2Vecで演算処理するWord2VecとニューラルネットワークWord2Vecの仕組みCBoWSkip-gramWord2Vecを応用することができる分野レコメンド機械翻訳Q&A・チャットボット感情分析Word2Vecの弱点Word2Vecの派生系や類似ツールGloVeWord

In [14]:
print(CreateAdditionalExplanation("out.txt"))

作ることがいかに大事かということがまああのわかるかと思います。あとは、これは中国語さっきまでできたのは英語の文法誤り訂正、何とか中国語とかロシア語だったんですが、これは中国語の誤り訂正で、でえーとまあ、単語ベクトルと言うものを三回位前やりましたよねと二勝で作ったと思うんですけど、最近、自然言語するで使われている単語ベクトルを作る手法は、文脈を考慮した単語ベクトルを作る手法が研究されていますと。
出ただ、これの入力の単語ベクトルをどういうふうに誤り訂正のシステムに入れるといいのかということが、最近までに解決の話題になっていて、ちょうど今年の今日か明日からうちの研究室のじいさんの金子くんという人がやった研究が投稿した当時、やばえっと世界最高精度だったんですけど。

['単語ベクトル', '誤り訂正', '中国語', '文法誤り訂正', '手法', '世界最高精度', '中国語', 'ロシア語', '研究室', '研究', 'あと', '英語', '位', '文脈', '入力', 'システム', '解決', '話題', 'じいさん', '金子', '人']
['単語ベクトル', '誤り訂正', '中国語', '文法誤り訂正', '手法', '世界最高精度', '中国語', 'ロシア語', '研究室', '研究', 'あと', '英語', '位', '文脈', '入力', 'システム', '解決', '話題', 'じいさん', '金子', '人']
[['単語ベクトル', 'Word2Vec：発明した本人も驚く単語ベクトルの驚異的な力 - DeepAge人工知能の今と一歩先を発信するメディアMENUすべてNumpyビッグデータ機械学習人工知能ディープラーニングTensorFlowホーム >ビッグデータ > Word2Vec：発明した本人も驚く単語ベクトルの驚異的な力採用情報アルバイト募集2016-09-2Word2Vec：発明した本人も驚く単語ベクトルの驚異的な力TweetWord2VecとはWord2Vecで演算処理するWord2VecとニューラルネットワークWord2Vecの仕組みCBoWSkip-gramWord2Vecを応用することができる分野レコメンド機械翻訳Q&A・チャットボット感情分析Word2Vecの弱点Word2Vecの派生系や類似ツールGloVeWord

In [19]:
text = "経路制御技術（ルーティング）とは、ネットワーク層が行う処理で目的のパケットの宛先のIPアドレスまでどのようにIPを経由して送られるかについて制御を行う技術である。\nインターネットにおける経路制御技術は、直接転送と間接転送がある。直接転送では、同一のネットワーク上のホストに転送を行い、ルータを経由する必要がない。一方間接転送は、異なるネットワーク上のホストへの転送を行う。\n経路選択には、事前に経路を決定する静的経路制御とルータ間でルーティングプロトコルを使用し、経路表を作成する動的経路制御の２種類がある。静的経路制御では、経路表が事前に作られているため、経路が安定する。しかし経路数が多くなった場合は設定が複雑になることやルータの障害の際に再設定が必要である。一方、動的経路制御はルータが経路を計算するため、経路計算の負荷や冗長性のある経路選択が行われるなどのデメリットがあるが、経路表を自動で作るため、障害迂回の際の経路変更が自動で行われるなどのメリットがある。今回の課題では、動的経路制御の一種であるOSPFのプロトコルについて調査を行う。\n"

importances = Extract_ImportantWords(text) #重要言葉の抽出
#print(importances)

['経路', '動的経路制御', '経路表', '経路制御技術', '静的経路制御', '経路選択', '転送', '制御', 'ルータ', '経路数', '経路計算', '経路変更', '間接転送', 'ネットワーク', 'ホスト', '事前', '設定', '自動', '技術', 'ルーティング', 'ネットワーク層', 'IPアドレス', 'IP', 'ルーティングプロトコル', '障害', '冗長性', '障害迂回', 'プロトコル', '処理', '目的', 'パケット', '宛先', 'インターネット', '同一', '種類', '負荷', 'デメリット', 'メリット', '課題', 'OSPF', '調査']
